# Multiple Non-Linear Regression

In this notebook we will build a multiple non-linear regression model, we will use polynomial features to make our original features higher order polynomials, enabling the model to capture non-linearity in the data.

## Outline

- [1 - Packages](#1)

- [2 - Dataset](#2)

- [3 - Multiple Non-Linear Regression Model Using Normal Equation](#3)
    - [3.1 Custom Model NE](#3point1)

<a id="1"></a>
## 1 - Packages

Below are the packages/libraries that we are going to use in this notebook.

In [1]:
# Importing necessary packages/libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

<a id="2"></a>
## 2 - Dataset

We will going to use dataset from [Kaggle](https://www.kaggle.com/datasets/hamzakhurshed/concrete-strength-dataset).

The dataset is downloaded and stored in `multiple_non_lr_dataset` folder. The folder contains a CSV file named **concrete_data.csv**, which we are going to use to train, validate and test our model.

Let's first load the dataset into pandas dataframe, and get the overview of it.

In [2]:
# Loading the dataset into pandas dataframe
data = pd.read_csv("multiple_non_lr_dataset/concrete_data.csv")
data

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


We have **1,030 rows** and **9 columns** in our dataset, from which we are going to split the dataset into **60%** for training, **20%** for cross-validation, and **20%** for testing. The first eight variables are feature variables $\bf{x}$, and our target variable $y$ is last variable.

While the dataset is clean, we will still going to look for any missing/NaN values in a dataset.

In [3]:
# Checking for any missing values in a dataset
data.isna().any()

Cement                False
Blast Furnace Slag    False
Fly Ash               False
Water                 False
Superplasticizer      False
Coarse Aggregate      False
Fine Aggregate        False
Age                   False
Strength              False
dtype: bool

There is no missing/NaN values in a dataset.

Let's convert the dataset into numpy arrays, and split it into training, cross-validation and testing sets.

In [4]:
# Converting pandas dataframe into numpy arrays
X = data.to_numpy()[:, 0:8]
Y = data.to_numpy()[:, 8:9]

print(f"Shape of X: {X.shape} & Shape of Y: {Y.shape}")

Shape of X: (1030, 8) & Shape of Y: (1030, 1)


In [5]:
# Splitting the dataset into training (60%) and temp (40%)
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.4, random_state=42)

# Splitting temp into validation (20%) and test (20%)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

print(f"Shape of X_train: {X_train.shape} & Shape of Y_train: {Y_train.shape}\n"
      f"Shape of X_val: {X_val.shape} & Shape of Y_val: {Y_val.shape}\n"
      f"Shape of X_test: {X_test.shape} & Shape of Y_test: {Y_test.shape}")

Shape of X_train: (618, 8) & Shape of Y_train: (618, 1)
Shape of X_val: (206, 8) & Shape of Y_val: (206, 1)
Shape of X_test: (206, 8) & Shape of Y_test: (206, 1)


<a id="3"></a>
## 3 - Multiple Non-Linear Regression Model Using Normal Equation

<a id="3point1"></a>
### 3.1 Custom Model NE

Since we do not know the data has linear or non-linear characteristics, we will train linear and few non-linear models, we will use **PolynomialFeatures** method from Scikit-Learn to transform $\bf{x}$ into higher-degree polynomial terms, capturing the non-linearity.

To find the optimal parameters for our model, we will use the **Gaussian Elimination method** to solve the below equation, instead of computing the inverse $(X^T \cdot X)^{-1}$ directly (which can lead to numerical instability).

$$ (X^T \cdot X) \cdot W = X^T \cdot Y \tag{1} $$

where,
- $W$ is a model parameters matrix.
- $X$ is a input feature matrix.
- $Y$ is a target variable matrix.

_**NOTE:** Solving regression problem using normal equation requires the bias term to be included in weight matrix. Therefore we also have to add 1's column vector to our original feature datasets (which will be automatically added when using **PolynomialFeatures** method), so the dimensions should be matched during matrix multiplication. And also feature scaling is not required because it does not use gradient descent to find the optimal parameters for the model._

In [6]:
# Dictionary to store models parameters, and polynomial transformers
models = {}

for degree in range(1, 6):  # Polynomial degrees 1 to 5 (degree 1 -> no polynomial features -> linear)

    # Applying polynomial features to training and validation data
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train)
    X_val_poly = poly.transform(X_val)

    # Finding model parameters using normal equation method
    W = np.linalg.solve((X_train_poly.T @ X_train_poly), (X_train_poly.T @ Y_train))
    
    # Evaluating on transformed training and cross-validation data
    Y_hat_train = X_train_poly @ W
    Y_hat_val = X_val_poly @ W

    # Calculating the cost
    train_cost = np.mean((Y_hat_train - Y_train)**2)
    val_cost = np.mean((Y_hat_val - Y_val)**2)

    print(f"Degree_{degree} model -> train_cost: {train_cost}, val_cost: {val_cost}\n")
 
    # Storing the model parameters, and polynomial transformer
    models[f"degree_{degree}"] = {
        "W": W,
        "poly": poly,
    }

Degree_1 model -> train_cost: 102.79385145348093, val_cost: 114.38470630338443

Degree_2 model -> train_cost: 49.537424005732774, val_cost: 61.950069371463734

Degree_3 model -> train_cost: 17.16449479097078, val_cost: 49.10177017537962

Degree_4 model -> train_cost: 4.110646166252671, val_cost: 1648597.3809762562

Degree_5 model -> train_cost: 1.8545894416303743, val_cost: 138618349056.97867



By seeing the above metrics, degree 2 is the best choice as it generalizes better. While Degree 3 has a lower training cost (**17.16** vs. **49.54** for Degree 2), its validation cost (**49.10**) is higher in range than Degree 2 (**61.95**), indicating possible overfitting. Degree 1 underfits (**train: 102.79, val: 114.38**), while Degrees 4 and 5 suffer from extreme overfitting. Thus, **Degree 2 offers the best balance** between bias and variance, making the dataset suitable for the task.

Now we will make prediction on testing dataset using the optimal model.

In [7]:
# Retrieving the optimal model parameters and polynomial transformer
W_2 = models["degree_2"]["W"]
poly_2 = models["degree_2"]["poly"]

In [8]:
# Applying polynomial transformation on test data
X_test_poly = poly_2.transform(X_test)

# Making prediction
Y_hat_test = X_test_poly @ W_2

# Calculating cost
test_cost = np.mean((Y_hat_test - Y_test)**2)
print(f"test_cost: {test_cost}")

test_cost: 64.99201321803871


The test cost shows good stability comparing with the validation cost, confirming that the model generalizes well to unseen data. This completes the task of building a **Multiple Non-Linear Regression Model Using Normal Equation**.